# **Crypto forecasting tutorial / 暗号通貨予測のチュートリアル**

参考:https://www.kaggle.com/junichiromorita/tutorial-to-the-g-research-crypto-competition-jpn

# G-Research Crypto forecasting competition / G-Research 暗号通貨予測コンペティション

In the [G-Research Crypto forecasting competition](link), participants have the challenge to predict  price returns across a bundle of major cryptocurrencies. To facilitate your participation, we have created this tutorial notebook covering some relevant concepts for the crypto forecasting challenge.

G-Research 暗号通貨予測コンペティションでは、参加者は主要な暗号通貨群の価格リターンを予測するという課題が与えれれます。参加しやすいように、このチュートリアルノートでは、暗号通貨予測コンペティションに関連するコンセプトを説明しています。

The notebook presents an introduction to crypto forecasting, describing the structure and elements of the dataset, some relevant statistical properties, as well as building a couple of ML baseline models and providing an example code submission.

このノートブックでは、暗号通貨予測について紹介し、データセットの構造と要素、関連する統計的特性を説明するとともに、いくつかの機械学習ベースラインモデルを構築し、コード投稿の例を示しています。

## The Cryptocurrency Market / 暗号通貨市場について

First, a quick introduction to the crypto world. Cryptocurrencies have become an extremely popular and volatile market, delivering massive returns (as well as losses) to investors. Thousands of cryptocurrencies have been created with a few major ones that many of you will have heard of including Bitcoin (BTC), Ether (ETH) or Dogecoin (DOGE).

まず、暗号通貨の世界について簡単に紹介します。暗号通貨は非常に人気が高く、変動の激しい市場となっており、投資家に多額のリターン（損失も含む）をもたらしています。何千もの暗号通貨が作られていますが、主なものとしては、ビットコイン（BTC）、イーサ（ETH）、ドージコイン（DOGE）などがあり、多くの人が聞いたことがあるでしょう。

Cryptocurrencies are traded extensively across crypto-exchanges, with an average volume of $41 billion traded daily over the last year, according to CryptoCompare (as of 25th July 2021). 

暗号通貨は暗号通貨取引所で広く取引されており、CryptoCompareによると、昨年1年間で毎日平均410億ドルの取引が行われています（2021年7月25日現在）。

Changes in prices between different cryptocurrencies are highly interconnected. For example, Bitcoin has historically been a major driver of price changes across cryptocurrencies but other coins also impact the market.  

異なる暗号通貨間の価格の変化は、高度に相互に関連しています。例えば、ビットコインは歴史的に暗号通貨間の価格変動の主な要因となってきましたが、他のコインも市場に影響を与えます。


## Forecasting returns / リターンの予測

A fundamental task in financial modeling is predicting how prices will behave in the near future. Using the time-series of historical prices as training data, we want to predict if prices will go up or down, and by how much, namely the asset *returns*.

金融モデリングの基本的な課題は、近い将来の価格がどのように推移するかを予測することです。過去の価格の時系列を学習データとして、価格が上がるのか下がるのか、どれくらい上がるのかを予測したい、つまり資産のリターンを予測したいのです。

In this competition, Kagglers are challenged to build machine learning models to predict the returns of 14 popular cryptocurrencies, in the time scale of minutes to hours. You will have access to millions of rows of minute-by-minute cryptocurrency trading data, with which you'll design your forecasting models for all 14 assets simultaneously. Your predictions will be evaluated by how much they correlate with real market data collected during the three-month evaluation period after the competition has closed. 

本コンペティションでは、人気のある14種類の暗号通貨のリターンを予測する機械学習モデルを、数分から数時間の時間スケールで構築することがKagglersの課題となっています。数百万行の分刻みの暗号通貨取引データにアクセスし、それをもとに14の資産すべてを同時に対象とした予測モデルを設計することになります。あなたの予測は、コンテスト終了後の3ヶ月間の評価期間中に収集された実際の市場データとどれだけ相関しているかによって評価されます。

Cryptocurrency returns prediction remains an open and extremely challenging forecasting task. This is a fascinating problem domain for the ML community given the extreme volatility of the assets, the non-stationary nature of the data, the market and meme manipulation, the correlation between assets and the very fast changing market conditions. We hope you find it as fascinating as we do! 

暗号通貨のリターン予測は、依然として未解決で、非常に困難な予測課題です。これは、資産の極端な変動性、データの非定常性、市場や操作、資産間の相関性、非常に速い市場状況の変化を考えると、機械学習コミュニティにとって魅力的な問題領域です。私たちと同じように、皆さんにも魅力的な問題を見つけていただきたいと思います。

# Dataset description / データセットの説明

Now, let's dive into the data! We start by loading the competition's dataset and inspecting its basic properties. 

それでは、データを見てみましょう。まずは、コンペティションのデータセットを読み込んで、その基本的なプロパティを確認します。

## Load the training set / トレーニングセットの読み込み

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime

In [ ]:
import lightgbm as lgbm

In [ ]:
data_folder = "../input/g-research-crypto-forecasting/"
!ls $data_folder

In [ ]:
crypto_df = pd.read_csv(data_folder + 'train.csv')

In [ ]:
crypto_df.head(10)

We can see that each row of the data set has the trading data for an asset, at a given minute timestamp, described in detail below. 

データセットの各行には、特定の分のタイムスタンプにおける資産の取引データがあることがわかります。

## Data features / データの特徴

We can see the different features included in the dataset. Specifically, the features included per asset are the following:

データセットに含まれる様々な特徴を見ることができます。具体的には、アセットごとに含まれている特徴は以下の通りです。

*   **timestamp**: All timestamps are returned as second Unix timestamps (the number of seconds elapsed since 1970-01-01 00:00:00.000 UTC). Timestamps in this dataset are multiple of 60, indicating minute-by-minute data. / すべてのタイムスタンプは、秒単位のUnixタイムスタンプ（1970-01-01 00:00:00.000 UTCからの経過秒数）で返されます。このデータセットのタイムスタンプは60の倍数で、1分ごとのデータを示しています。
*   **Asset_ID**: The asset ID corresponding to one of the crytocurrencies (e.g. `Asset_ID = 1` for Bitcoin). The mapping from `Asset_ID` to crypto asset is contained in `asset_details.csv`. / いずれかの暗号通貨に対応するアセットID（例：Bitcoinの場合は`Asset_ID = 1`）。`Asset_ID`から暗号資産へのマッピングは`asset_details.csv`に含まれる。
*   **Count**: Total number of trades in the time interval (last minute). / 時間間隔（最後の1分）での取引の総数。
*   **Open**:	Opening price of the time interval (in USD). / 時間間隔内の始値（単位：米ドル）。
*   **High**:	Highest price reached during time interval (in USD). / 時間間隔内の高値（単位：米ドル）。
*   **Low**: Lowest price reached during time interval (in USD). / 時間間隔内の安値（単位：米ドル）。
*   **Close**:	Closing price of the time interval (in USD). / 時間間隔内の終値（米ドル）。
*   **Volume**:	Quantity of asset bought or sold, displayed in base currency USD. / 買ったまたは売った資産の量で、基本通貨の米ドルで表示されます。
*   **VWAP**: The average price of the asset over the time interval, weighted by volume. VWAP is an aggregated form of trade data. / 取引時間中の平均価格を取引量で加重平均したもの。VWAPは取引データを集計したものです。
*   **Target**: Residual log-returns for the asset over a 15 minute horizon. / 15分間の暗号資産の残余対数リターン。

The first two columns define the time and asset indexes for this data row. The 6 middle columns are feature columns with the trading data for this asset and minute in time. The last column is prediction target, which we will get to later in more detail.

最初の2列は、このデータ行の時間と資産のインデックスを定義しています。中央の6列は、この資産と時間の分の取引データを含む特徴的な列です。最後の列は予測ターゲットですが、これについては後で詳しく説明します。

We also view the asset information, including the list of all assets, the `Asset_ID` to asset mapping, and the weight of each asset used to weigh their relative importance in the evaluation metric.

また、すべてのアセットのリスト、アセット_IDとアセットのマッピング、評価指標における相対的な重要性を評価するために使用される各アセットの重みなど、アセット情報も表示されます。

In [ ]:
asset_details = pd.read_csv(data_folder + 'asset_details.csv')
asset_details

# Preprocessing / 前処理

## Dealing with missing data / 欠損データの処理


Let us inspect the data for another important asset, Ethereum.

もう一つの重要な暗号資産であるEthereumのデータを調べてみましょう。

In [ ]:
crypto_df['date'] = crypto_df['timestamp'].apply(lambda x: datetime.fromtimestamp(x))

In [ ]:
# NaNとなっているTargetを学習データから除外した
crypto_df = crypto_df[(crypto_df['Target']>0)|(crypto_df['Target']<=0)]

## Data visualisation / データの可視化

We  will start by visualising the Close prices for the two assets we have selected.

選択した2つの暗号資産の終値を可視化してみます。

In [ ]:
# import matplotlib.pyplot as plt

# # plot vwap time series for both chosen assets
# f = plt.figure(figsize=(15,4))

# # fill missing values for BTC
# btc = btc.reindex(range(btc.index[0],btc.index[-1]+60,60),method='pad')

# ax = f.add_subplot(121)
# plt.plot(btc['Close'], label='BTC')
# plt.legend()
# plt.xlabel('Time')
# plt.ylabel('Bitcoin')

# ax2 = f.add_subplot(122)
# ax2.plot(eth['Close'], color='red', label='ETH')
# plt.legend()
# plt.xlabel('Time')
# plt.ylabel('Ethereum')

# plt.tight_layout()
# plt.show()

The assets have quite different history, but we could check if they correlate in recent times.

両暗号資産の歴史は全く異なりますが、最近では相関関係があることを確認できました。

In [ ]:
# import time

# # auxiliary function, from datetime to timestamp
# totimestamp = lambda s: np.int32(time.mktime(datetime.strptime(s, "%d/%m/%Y").timetuple()))

# # create intervals
# btc_mini_2021 = btc.loc[totimestamp('01/06/2021'):totimestamp('01/07/2021')]
# eth_mini_2021 = eth.loc[totimestamp('01/06/2021'):totimestamp('01/07/2021')]

In [ ]:
# # plot time series for both chosen assets
# f = plt.figure(figsize=(7,8))

# ax = f.add_subplot(211)
# plt.plot(btc_mini_2021['Close'], label='btc')
# plt.legend()
# plt.xlabel('Time')
# plt.ylabel('Bitcoin Close')

# ax2 = f.add_subplot(212)
# ax2.plot(eth_mini_2021['Close'], color='red', label='eth')
# plt.legend()
# plt.xlabel('Time')
# plt.ylabel('Ethereum Close')

# plt.tight_layout()
# plt.show()

On shorter intervals we can visually see some potential correlation between both assets, with some simultaneous ups and downs. A better format for analyzing such movements is by calculating asset returns. 

もっと短い間隔で見ると、両暗号資産の間には潜在的な相関関係があり、同時にいくつかの上昇と下降があることが視覚的にわかります。このような動きを分析するには、資産のリターンを計算するのが良いでしょう。

# Building your prediction model / 予測モデルの構築

## Prediction targets and evaluation / 予測対象と評価

This forecasting competition aims to predict returns in the near future for prices $P^a$, for each asset $a$. For each row in the dataset, we include the target for prediction, `Target`. `Target` is derived from log returns ($R^a$) over 15 minutes.

この予測コンペティションは、各資産$a$について、価格$P^a$に対する近未来のリターンを予測することを目的としています。データセットの各行には、予測のターゲットである`Target`が含まれる。`Target`は15分間の対数収益率($R^a$)から得られる。

$$R^a(t) = log (P^a(t+16)\ /\ P^a(t+1))$$

Crypto asset returns are highly correlated, following to a large extend the overall crypto market. As we want to test your ability to predict returns for individual assets, we perform a linear residualization, removing the market signal from individual asset returns when creating the two targets. In more detail, if $M(t)$ is the weighted average market returns, the target is:

暗号資産のリターンは非常に相関性が高く、暗号市場全体を大きくフォローしています。個々の資産のリターンを予測する能力をテストしたいため、2つのターゲットを作成する際には、個々の資産のリターンから市場のシグナルを除去し、線形残差を行います。より詳細には、$M(t)$が加重平均市場リターンである場合、ターゲットは

$$M(t) = \frac{\sum_a w^a R^a(t)}{\sum_a w^a}  \\
\beta^a = \frac{\langle M \cdot R^a \rangle}{\langle M^2 \rangle} \\
\text{Target}^a(t) = R^a(t) - \beta^a M(t)$$

where the bracket $\langle .\rangle$ represent the rolling average over time (3750 minute windows). 

ここで、括弧内の$\langle .\rangle$は、時間（3750分窓）でのローリング平均を表しています。

Some rows have null values for targets due to missing values in future prices. Rows with nulls in the test set ground truth are ignored for scoring purposes.

一部の行では、将来の価格の値が欠落しているため、ターゲットの値が欠損値になっています。テストセットのに欠損値がある行は、スコアリングのために無視されます。

In the competition, your predictions will be evaluated on a weighted version of the Pearson correlation coefficient, with weights given by the `Weight` column in the Asset Details file.

コンペティションでは、あなたの予測は、ピアソン相関係数の加重バージョンで評価されます。加重は、資産詳細ファイルの Weight 列で与えられます。

In this tutorial, we will simplify things and use correlation (without weights) for evaluation, and consider only two assets, BTC and ETH.

このチュートリアルでは、物事を単純化して、相関関係（重みなし）を評価に使用し、BTC と ETH の 2 つの資産のみを検討します。

## Feature design / 特徴量設計

We first design a few relevant features to input to our model.

まず、モデルに入力するための関連する特徴量をいくつか設計します。

In [ ]:
# # Select some input features from the trading data: 
# # 5 min log return, abs(5 min log return), upper shadow, and lower shadow.
# upper_shadow = lambda asset: asset.High - np.maximum(asset.Close,asset.Open)
# lower_shadow = lambda asset: np.minimum(asset.Close,asset.Open)- asset.Low

# X_btc = pd.concat([log_return(btc.VWAP,periods=5), log_return(btc.VWAP,periods=1).abs(), 
#                upper_shadow(btc), lower_shadow(btc)], axis=1)
# y_btc = btc.Target

# X_eth = pd.concat([log_return(eth.VWAP,periods=5), log_return(eth.VWAP,periods=1).abs(), 
#                upper_shadow(eth), lower_shadow(eth)], axis=1)
# y_eth = eth.Target

## Preparing the data for building predictive models / 予測モデル構築のためのデータの準備

As we will train linear regression parameters, we need to separate training and test sets. To do so, we will compute X and y and split this data into train and test splits. Note that the test split represents a later part of the data, as it is commonly done in time series. 

線形回帰パラメータをトレーニングするので、トレーニングセットとテストセットを分ける必要があります。そのために、XとYを計算し、このデータを訓練用とテスト用に分割します。時系列でよく行われるように、テスト分割はデータの後の部分を使用していることに注意してください。

In [ ]:
crypto_df.tail()

In [ ]:
crypto_df['Target'].max()

In [ ]:
crypto_df['Target'].min()

In [ ]:
# select training and test periods
# 時系列データなので、testデータは未来になるように調整
train_term_end = '2021-03-01'

In [ ]:
asset_details

In [ ]:
crypto_df.head()

In [ ]:
datamart= pd.merge(crypto_df,asset_details[['Asset_ID','Weight']],on='Asset_ID',how='left')

In [ ]:
# datamart.head()

In [ ]:
# divide data into train and test, compute X and y
# 時系列データなので、testデータは未来になるように調整
lgbm_train = datamart[datamart['date']< train_term_end].fillna(0)
lgbm_eval = datamart[datamart['date']>= train_term_end].fillna(0)

In [ ]:
# lgbm_train.head()

In [ ]:
drop_list = ['Target','date']

In [ ]:
y_train = lgbm_train['Target']
X_train = lgbm_train.drop(drop_list , axis=1)
y_test = lgbm_eval['Target']
X_test = lgbm_eval.drop(drop_list , axis=1)


dataset_train=lgbm.Dataset(X_train, y_train)
dataset_eval=lgbm.Dataset(X_test, y_test, reference=dataset_train)

In [ ]:
# X_train.head()

In [ ]:
params = {'task': 'train',
          'boosting_type': 'gbdt',      # GBDTを指定
          'objective': 'regression', # 目的 : 回帰  
          'metric': {'rmse'}, # 評価指標 : rsme(平均二乗誤差の平方根) 
          'learning_rate': 0.1,         # 学習率
          'num_leaves': 23,             # ノードの数
          'min_data_in_leaf': 3,        # 決定木ノードの最小データ数
          'num_iteration': 1000         # 予測器(決定木)の数:イタレーション
         }
 

In [ ]:
# モデルの学習
lgbm_model = lgbm.train(params,
                        train_set=dataset_train, # トレーニングデータの指定
                        valid_sets=dataset_eval, # 検証データの指定
                        early_stopping_rounds=1000
                  )


In [ ]:
y_pred = lgbm_model.predict(X_test)

In [ ]:
lgbm.plot_importance(lgbm_model, figsize=(12, 8))


## Evaluate baselines / ベースラインの評価

The competition performance metric is weighted correlation. However, for now we will use simple correlation to evaluate the two baseline models built.

コンペティションの評価指標は加重相関です。しかし、ここでは単純相関を用いて、構築した2つのベースラインモデルを評価します。

In [ ]:
from sklearn.metrics import mean_squared_error # モデル評価用(平均二乗誤差)
from sklearn.metrics import r2_score # モデル評価用(決定係数)
# モデル評価
# rmse : 平均二乗誤差の平方根
mse = mean_squared_error(y_test, y_pred) # MSE(平均二乗誤差)の算出
rmse = np.sqrt(mse) # RSME = √MSEの算出
print('RMSE :',rmse)

#r2 : 決定係数
r2 = r2_score(y_test,y_pred)
print('R2 :',r2)

## Submission / 提出方法

Note that this is a Code Competition, in which you must submit your notebook to be run against the hidden private data. Your notebook should use the provided python time-series API, which ensures that models do not peek forward in time. To use the API, follow the instructions and template in [Code Competition Detailed API instructions](https://www.kaggle.com/sohier/detailed-api-introduction) and [Basic Submission Template](https://www.kaggle.com/sohier/basic-submission-template).

これはコードコンペティションであり、隠されたプライベートデータに対して実行されるノートブックを提出する必要があります。ノートブックは、提供されているpython時系列APIを使用する必要があります。これにより、モデルが時間的に先に進むことがないようになっています。APIを使用するには、[Code Competition Detailed API instructions](https://www.kaggle.com/sohier/detailed-api-introduction)と[Basic Submission Template](https://www.kaggle.com/sohier/basic-submission-template)の指示とテンプレートに従ってください。